In [1]:
import tensorflow as tf
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [0]:
def comp(h):
  plt.title('loss/mse')
  plt.plot(h.history['loss'], label='train')
  plt.plot(h.history['val_loss'], label='test')
  plt.legend()
  plt.show()

In [0]:
df = pd.read_csv('housing.csv', header=None)

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [0]:
x = df.drop(columns=13)
y = df[13]

In [7]:
x.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33


In [8]:
y.head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: 13, dtype: float64

In [9]:
df[8].unique()

array([ 1,  2,  3,  5,  4,  8,  6,  7, 24])

In [10]:
df.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

In [11]:
cv_scores = []
kfold = KFold(n_splits=10, shuffle=True, random_state=1)
x = np.array(x)
y = np.array(y)

for train, test in kfold.split(x, y):
  #buat arsitektur jaringan
  model = tf.keras.Sequential()

  #tambah layer (input layer)
  model.add(tf.keras.layers.Dense(512, activation='relu', input_shape=(13,)))

  #tambah layer (hidden layer)
  model.add(tf.keras.layers.Dense(256, activation='relu'))
  model.add(tf.keras.layers.Dense(128, activation='relu'))
  model.add(tf.keras.layers.Dense(64, activation='relu'))
  model.add(tf.keras.layers.Dense(32, activation='relu'))

  #tambah layer (output layer)
  model.add(tf.keras.layers.Dense(1, activation='linear'))

  #compile model
  model.compile(loss='mse', optimizer='adam', metrics=['mse'])
  
  model.fit(x[train], y[train], epochs=100, batch_size=10, verbose=0)
  scores = model.evaluate(x[test], y[test], verbose=0)
  cv_scores.append(scores[1])
  print(f'nilai msenya adalah :{scores[1]}')
print(f'nilai Mean +/- {np.mean(cv_scores)}')
print(f'nilai Stdev +/- {np.std(cv_scores)}')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
nilai msenya adalah :15.452154159545898
nilai msenya adalah :12.261883735656738
nilai msenya adalah :14.10354995727539
nilai msenya adalah :33.049560546875
nilai msenya adalah :18.98746681213379
nilai msenya adalah :14.324466705322266
nilai msenya adalah :12.199670791625977
nilai msenya adalah :23.4776554107666
nilai msenya adalah :37.36949157714844
nilai msenya adalah :12.265375137329102
nilai Mean +/- 19.34912872314453
nilai Stdev +/- 8.654963493347168


In [12]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [0]:
scaler = StandardScaler()

In [0]:
df_2 = df.copy()

In [0]:
features = [0,1,2,4,5,6,7,8,9,10,11,12]
for feature in features:
  df_2[[feature]] = scaler.fit_transform(df_2[[feature]])

In [18]:
cv_scores = []
kfold = KFold(n_splits=10, shuffle=True, random_state=1)
x = df_2.drop(columns=13)
y = df_2[13]
x = np.array(x)
y = np.array(y)

for train, test in kfold.split(x, y):
  #buat arsitektur jaringan
  model = tf.keras.Sequential()

  #tambah layer (input layer)
  model.add(tf.keras.layers.Dense(512, activation='relu', input_shape=(13,)))

  #tambah layer (hidden layer)
  model.add(tf.keras.layers.Dense(256, activation='relu'))
  model.add(tf.keras.layers.Dense(128, activation='relu'))
  model.add(tf.keras.layers.Dense(64, activation='relu'))
  model.add(tf.keras.layers.Dense(32, activation='relu'))

  #tambah layer (output layer)
  model.add(tf.keras.layers.Dense(1, activation='linear'))

  #compile model
  model.compile(loss='mse', optimizer='adam', metrics=['mse'])
  
  model.fit(x[train], y[train], epochs=100, batch_size=10, verbose=0)
  scores = model.evaluate(x[test], y[test], verbose=0)
  cv_scores.append(scores[1])
  print(f'nilai msenya adalah :{scores[1]}')
print(f'nilai mean +/- {np.mean(cv_scores)}')
print(f'nilai stdev +/- {np.std(cv_scores)}')

nilai msenya adalah :9.423498153686523
nilai msenya adalah :10.822734832763672
nilai msenya adalah :6.332294464111328
nilai msenya adalah :22.733095169067383
nilai msenya adalah :4.890323162078857
nilai msenya adalah :8.617108345031738
nilai msenya adalah :7.169856071472168
nilai msenya adalah :13.705907821655273
nilai msenya adalah :11.706348419189453
nilai msenya adalah :6.937340259552002
nilai mean +/- 10.233850479125977
nilai stdev +/- 4.879572868347168


In [0]:
#data jumlahnya menjadi 374
new_df = df_2[df_2[8] < 24]

In [21]:
cv_scores = []
kfold = KFold(n_splits=10, shuffle=True, random_state=1)
x = df_2.drop(columns=13)
y = df_2[13]
x = np.array(x)
y = np.array(y)

for train, test in kfold.split(x, y):
  #buat arsitektur jaringan
  model = tf.keras.Sequential()

  #tambah layer (input layer)
  model.add(tf.keras.layers.Dense(512, activation='relu', input_shape=(13,)))

  #tambah layer (hidden layer)
  model.add(tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.5)))
  model.add(tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.5)))
  model.add(tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.5)))
  model.add(tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.5)))

  #tambah layer (output layer)
  model.add(tf.keras.layers.Dense(1, activation='linear'))

  #compile model
  model.compile(loss='mse', optimizer='adam', metrics=['mse'])
  
  model.fit(x[train], y[train], epochs=100, batch_size=10, verbose=0)
  scores = model.evaluate(x[test], y[test], verbose=0)
  cv_scores.append(scores[1])
  print(f'nilai msenya adalah :{scores[1]}')
print(f'nilai mean +/- {np.mean(cv_scores)}')
print(f'nilai stdev +/- {np.std(cv_scores)}')

nilai msenya adalah :8.906962394714355
nilai msenya adalah :7.423862457275391
nilai msenya adalah :8.78817367553711
nilai msenya adalah :29.41681480407715
nilai msenya adalah :6.7900390625
nilai msenya adalah :10.823613166809082
nilai msenya adalah :10.372838973999023
nilai msenya adalah :9.223105430603027
nilai msenya adalah :16.02733612060547
nilai msenya adalah :8.483141899108887
nilai mean +/- 11.625588417053223
nilai stdev +/- 6.405009746551514
